# FROM CSV TO PDF

- need to add code that will immediately add and vectorise the pdf to the db, dont want to have a pdf saved
- test5

In [1]:
import pandas as pd
import bs4
from langchain_community.document_loaders import WebBaseLoader
from fpdf import FPDF
import uuid
import unidecode

# Read the CSV file
file_name = 'news_data_test.csv'
df = pd.read_csv(file_name)

path = './data3/'
bs4_strainer = bs4.SoupStrainer(class_=("o-topper__headline o-topper__headline--large", "n-content-body js-article__content-body"))
pdfs_counter = 0

for index, row in df.iterrows():
    # Extract the relevant fields from the row
    heading = row['Headline']
    href_tag = row['Link']
    date = row['Date']
    
    # Extract the href link from the href_tag
    soup = bs4.BeautifulSoup(href_tag, 'html.parser')
    href = soup.find('a')['href']
    
    myurl = 'https://www.ft.com' + href
    
    # Load the content from the URL
    loader = WebBaseLoader(
        web_paths=(myurl,),
        bs_kwargs={"parse_only": bs4_strainer},
    )
    docs = loader.load()
    
    content = docs[0].page_content
    
    heading1 = unidecode.unidecode(heading)
    date1 = unidecode.unidecode(date)
    content = unidecode.unidecode(content)

    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.cell(200, 10, txt="Title: " + heading1, ln=True)
    pdf.cell(200, 10, txt="Date: " + date1, ln=True)
    pdf.multi_cell(0, 10, txt=content)  # Use multi_cell for content to handle line breaks

    # Generate a unique file name
    file_name = str(uuid.uuid4()) + ".pdf"
    full_path = path + file_name

    # Output PDF for this article
    pdf.output(full_path)
    pdfs_counter += 1
    
print(f"{pdfs_counter} files saved to data3 folder.")


USER_AGENT environment variable not set, consider setting it to identify your requests.


3 files saved to data3 folder.
